In [1]:
import jax.numpy as jnp
import numpy as np
import time
import chex
import jax
import gymnasium as gym
import exciting_environments as excenv
import random
import matplotlib.pyplot as plt
import diffrax

### Simulate Ahead

In [2]:
env=excenv.MassSpringDamper(batch_size=5,action_constraints={"force"=[10,20,30,40,50]})

In [3]:
acts=jnp.full((5,8),jnp.array([1,1,1,1,1,1,1,1])*10)[:,:,None]

In [4]:
state=env.vmap_init_state()
obs,states,last_state=env.vmap_sim_ahead(state,
                   acts,
                   env.tau,
                   env.tau
                   )
obs[0]

Array([[0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 2.0000001e-03],
       [2.0000000e-07, 3.9998000e-03],
       [5.9997996e-07, 5.9993980e-03],
       [1.1999199e-06, 7.9987925e-03],
       [1.9997988e-06, 9.9979797e-03],
       [2.9995970e-06, 1.1996960e-02],
       [4.1992930e-06, 1.3995731e-02],
       [5.5988658e-06, 1.5994290e-02]], dtype=float32, weak_type=True)

In [55]:
k=[]
def test(x):
    return x.physical_state.deflection

In [56]:
states

ClassicCoreEnvironment.State(physical_state=MassSpringDamper.PhysicalState(deflection=Array([[0.0000000e+00, 0.0000000e+00, 2.0000000e-06, 5.9997992e-06,
        1.1999198e-05, 1.9997988e-05, 2.9995968e-05, 4.1992927e-05,
        5.5988658e-05],
       [0.0000000e+00, 0.0000000e+00, 2.0000000e-06, 5.9997992e-06,
        1.1999198e-05, 1.9997988e-05, 2.9995968e-05, 4.1992927e-05,
        5.5988658e-05],
       [0.0000000e+00, 0.0000000e+00, 2.0000000e-06, 5.9997992e-06,
        1.1999198e-05, 1.9997988e-05, 2.9995968e-05, 4.1992927e-05,
        5.5988658e-05],
       [0.0000000e+00, 0.0000000e+00, 2.0000000e-06, 5.9997992e-06,
        1.1999198e-05, 1.9997988e-05, 2.9995968e-05, 4.1992927e-05,
        5.5988658e-05],
       [0.0000000e+00, 0.0000000e+00, 2.0000000e-06, 5.9997992e-06,
        1.1999198e-05, 1.9997988e-05, 2.9995968e-05, 4.1992927e-05,
        5.5988658e-05]], dtype=float32, weak_type=True), velocity=Array([[0.        , 0.02      , 0.039998  , 0.05999398, 0.07998792,
    

In [57]:
jax.vmap(test, in_axes=(0))(state)

In [58]:
env.generate_rew_trunc_term_ahead(states,acts)

ValueError: vmap got inconsistent sizes for array axes to be mapped:
  * most axes (4 of them) had size 4, e.g. axis 0 of argument state[<flat index 0>][<flat index 0>] of type float32[4,9];
  * one axis had size 40: axis 0 of argument action of type int32[40,1]

In [62]:
state

ClassicCoreEnvironment.State(physical_state=MassSpringDamper.PhysicalState(deflection=Array([0, 0, 0, 0, 0], dtype=int32, weak_type=True), velocity=Array([0, 0, 0, 0, 0], dtype=int32, weak_type=True)), PRNGKey=None, additions=None, reference=MassSpringDamper.PhysicalState(deflection=Array([nan, nan, nan, nan, nan], dtype=float32, weak_type=True), velocity=Array([nan, nan, nan, nan, nan], dtype=float32, weak_type=True)))

In [9]:
state

ClassicCoreEnvironment.State(physical_state=MassSpringDamper.PhysicalState(deflection=Array([0, 0, 0, 0, 0], dtype=int32, weak_type=True), velocity=Array([0, 0, 0, 0, 0], dtype=int32, weak_type=True)), PRNGKey=None, additions=None, reference=MassSpringDamper.PhysicalState(deflection=Array([nan, nan, nan, nan, nan], dtype=float32, weak_type=True), velocity=Array([nan, nan, nan, nan, nan], dtype=float32, weak_type=True)))

In [15]:
obs.shape

(5, 9, 2)

In [16]:
acts.shape

(5, 8, 1)

In [11]:
state=env.vmap_init_state()
for _ in range(acts.shape[1]):
    obs,state=env.vmap_step(state=state,action=10*jnp.ones(5).reshape(-1,1))
    print(obs[0,:])

[0.    0.002   nan   nan]
[2.0000e-07 3.9998e-03        nan        nan]
[5.9997996e-07 5.9993980e-03           nan           nan]
[1.1999198e-06 7.9987925e-03           nan           nan]
[1.999799e-06 9.997981e-03          nan          nan]
[2.9995970e-06 1.1996961e-02           nan           nan]
[4.1992930e-06 1.3995731e-02           nan           nan]
[5.598866e-06 1.599429e-02          nan          nan]
